In [1]:
import torch
import torchvision
import torchaudio

import cv2
from PIL import Image

import librosa
import soundfile as sf

from transformers import AutoModel, AutoTokenizer, AutoProcessor

import moviepy.editor as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

import matplotlib.pyplot as plt
from tqdm import tqdm

!pip install ffmpeg-python
!pip install json5
import json5
import sys

!pip install pytube
from pytube import YouTube
!pip install ffmpeg-python
import ffmpeg

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.0 MB/s eta 0:00:00


In [11]:
import os
folders = [
    "Data/videos",
    "Data/audio",
    "Data/frames",
    "Data/spectrograms",
    "Models/video_model",
    "Models/audio_model",
    "Models/multimodal",
    "Data/text_features"
]

for f in folders:
  os.makedirs(f, exist_ok=True)

print("Folder structure created successfully")

Folder structure created successfully


In [49]:
import os
from google.colab import files


input_folder = "Data/videos"
os.makedirs(input_folder, exist_ok=True)


uploaded = files.upload()


for filename in uploaded.keys():
    with open(os.path.join(input_folder, filename), "wb") as f:
        f.write(uploaded[filename])


Saving WIN_20251223_22_25_20_Pro.mp4 to WIN_20251223_22_25_20_Pro.mp4


In [51]:
#Extract frames from videos & images

input_folder = "Data/videos"
output_folder = "Data/frames"


os.makedirs(output_folder, exist_ok=True)


FRAME_RATE = 1


def extract_frames_from_video(video_path, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_interval = int(fps / FRAME_RATE) if fps > 0 else 1
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break


        if frame_count % frame_interval == 0:
            frame_path = os.path.join(save_dir, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_count += 1

        frame_count += 1

    cap.release()

def copy_image(img_path, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    img = Image.open(img_path)
    if img.mode == 'RGBA':
        img = img.convert('RGB')
    img.save(os.path.join(save_dir, "image_0001.jpg"))


for file_name in tqdm(os.listdir(input_folder), desc="Processing files"):
    file_path = os.path.join(input_folder, file_name)
    save_directory = os.path.join(output_folder, os.path.splitext(file_name)[0])

    if file_name.lower().endswith((".mp4", ".avi", ".mov", ".mkv", ".flv")):
        print(f"\nExtracting frames from video: {file_name}")
        extract_frames_from_video(file_path, save_directory)

    elif file_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
        print(f"\nCopying image: {file_name}")
        copy_image(file_path, save_directory)

    else:
        print(f"\nSkipping Unsupported Files: {file_name}")

print("\nFrame extraction successfully!")

Processing files:   0%|          | 0/20 [00:00<?, ?it/s]


Skipping Unsupported Files: DDD_samples.gif

Extracting frames from video: video_file.mp4


Processing files:  10%|█         | 2/20 [00:04<00:37,  2.07s/it]


Skipping Unsupported Files: deepfakes.gif

Copying image: ex_deepfakes.png

Copying image: ex_original.png

Skipping Unsupported Files: neuraltextures.gif

Extracting frames from video: WIN_20251223_22_25_20_Pro.mp4


Processing files: 100%|██████████| 20/20 [00:09<00:00,  2.03it/s]


Skipping Unsupported Files: faceshifter.gif

Skipping Unsupported Files: face2face.gif

Copying image: ex_deepfakes_mask.png

Copying image: ex_deepfakedetection.png

Skipping Unsupported Files: deepfakedetection.gif

Copying image: table4_faceshifter.png

Skipping Unsupported Files: faceswap.gif

Copying image: ex_neuraltextures_mask.png

Copying image: ex_original_actors.png

Copying image: teaser.png

Copying image: ex_neuraltextures.png

Copying image: ex_deepfakedetection_mask.png

Skipping Unsupported Files: .ipynb_checkpoints

Frame extraction successfully!


In [52]:
from librosa.feature import chroma_stft
# Extract AUdio and Audio Features

from moviepy.editor import VideoFileClip

video_folder = "Data/videos"
audio_folder = "Data/audio"
feature_folder = "Data/audio_features"

os.makedirs(audio_folder, exist_ok = True)
os.makedirs(feature_folder, exist_ok = True)

def extract_audio(video_path, save_path):
  video = VideoFileClip(video_path)
  audio = video.audio
  audio.write_audiofile(save_path, fps =  16000)
  video.close()

def extract_audio_features(audio_path):
  y, sr = librosa.load(audio_path, sr = 16000)

  features = {
      "mfcc" : librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20),
      "mel"  : librosa.feature.melspectrogram(y=y, sr=sr),
      "chroma": librosa.feature.chroma_stft(y=y, sr=sr)
  }
  return features

for file in tqdm(os.listdir(video_folder), desc = "Processing Videos"):
  if file.lower().endswith((".mp4", ".avi", ".mov", ".mkv", ".flv")):
    video_path = os.path.join(video_folder, file)
    base_name = os.path.splitext(file)[0]
    audio_output_path = os.path.join(audio_folder, base_name + ".wav")


    print(f"\n Extracting audio from: {file}")
    extract_audio(video_path, audio_output_path)

    print("Extracting audio features..")
    features = extract_audio_features(audio_output_path)


    feature_save_path = os.path.join(
        feature_folder, base_name + ".npy"
    )
    np.save(feature_save_path, features)

print("\n Audio and Audio Features extracted succesfully")

Processing Videos:   0%|          | 0/20 [00:00<?, ?it/s]


 Extracting audio from: video_file.mp4


Processing Videos:   0%|          | 0/20 [00:00<?, ?it/s]

MoviePy - Writing audio in Data/audio/video_file.wav



Processing Videos:   0%|          | 0/20 [00:01<?, ?it/s]

...

MoviePy - Writing audio in Data/audio/WIN_20251223_22_25_20_Pro.wav



Processing Videos:  10%|█         | 2/20 [00:03<00:26,  1.48s/it]

MoviePy - Done.
Extracting audio features..


Processing Videos: 100%|██████████| 20/20 [00:04<00:00,  4.88it/s]


 Audio and Audio Features extracted succesfully


In [53]:
# Speech to text + tetx feature extraction
!pip install git+https://github.com/openai/whisper.git
!pip install transformers sentencepiece

import whisper


audio_folder = "Data/audio"
text_folder  = "Data/text"
text_features_folder = "Data/text_features"


os.makedirs(text_folder, exist_ok = True)
os.makedirs(text_features_folder, exist_ok = True)

model = whisper.load_model("base")

embed_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embed_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_sentence_embedding(text):
  tokens = embed_tokenizer(text, return_tensors = "pt", truncation = True, padding = True)
  with torch.no_grad():
    output = embed_model(**tokens)
  return output.last_hidden_state.mean(dim = 1).numpy()

for file in tqdm(os.listdir(audio_folder), desc = "Speech_To_Text"):
  if file.lower().endswith(".wav"):

    audio_path = os.path.join (audio_folder, file)
    text_output_path = os.path.join(text_folder, file.replace(".wav", ".txt"))
    feature_output = os.path.join(text_features_folder, file.replace(".wav", ".npy" ))

    print(f"\n Transcribing: {file}")
    result = model.transcribe(audio_path)
    text = result["text"]

    with open(text_output_path, "w") as f:
      f.write(text)

    print(" Extracting text features...")
    embedding = get_sentence_embedding(text)

    np.save(feature_output, embedding)

print("\n Speech_To_Text + Text_ Features Exttracted")

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-uw5z6zua
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-uw5z6zua
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Speech_To_Text:   0%|          | 0/2 [00:00<?, ?it/s]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")




 Transcribing: WIN_20251223_22_25_20_Pro.wav
 Extracting text features...


Speech_To_Text:  50%|█████     | 1/2 [00:15<00:15, 15.27s/it]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")




 Transcribing: video_file.wav


Speech_To_Text: 100%|██████████| 2/2 [00:20<00:00, 10.12s/it]

 Extracting text features...

 Speech_To_Text + Text_ Features Exttracted


In [54]:
# Import required libraries
import cv2  # OpenCV for image processing
import dlib  # For face detection and alignment
import os   # For handling file paths

# Check if the dlib shape predictor file exists, if not, download it
predictor_path = 'shape_predictor_68_face_landmarks.dat'
if not os.path.exists(predictor_path):
    print(f"Downloading {predictor_path}...")
    # This URL is for the 68-point face landmark detector from dlib's github
    !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
    !bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2 # Decompress

# Initialize face detector and shape predictor
detector = dlib.get_frontal_face_detector()  # Detects faces in an image
predictor = dlib.shape_predictor(predictor_path)
# Pre-trained model to locate 68 facial landmarks for alignment

# Function to crop and align face from an image
def crop_align_face(image_path, output_path):
    # Read image
    img = cv2.imread(image_path)

    # Convert image to grayscale (dlib works better on grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector(gray)

    # Loop through detected faces
    for i, face in enumerate(faces):
        # Get landmarks for the face
        landmarks = predictor(gray, face)

        # Extract the coordinates for eyes (for alignment)
        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)

        # Calculate angle between eyes
        dy = right_eye[1] - left_eye[1]
        dx = right_eye[0] - left_eye[0]
        angle = cv2.fastAtan2(dy, dx)

        # Compute center of eyes
        eyes_center = ((left_eye[0] + right_eye[0]) // 2,
                       (left_eye[1] + right_eye[1]) // 2)

        # Get rotation matrix for alignment
        M = cv2.getRotationMatrix2D(eyes_center, angle, 1.0)

        # Rotate the image to align eyes horizontally
        aligned_face = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

        # Crop the face using the original face rectangle
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cropped_face = aligned_face[y:y+h, x:x+w]

        # Save cropped and aligned face
        os.makedirs(output_path, exist_ok=True)
        cv2.imwrite(os.path.join(output_path, f'face_{i}.jpg'), cropped_face)

In [55]:
# AUdio MOdel
import torch
import torch.nn as nn

# The AudioModel class definition has been moved to cell W-lU2qGHJYXT
# This cell is now empty or can be removed if not needed for other purposes.

In [56]:
#Text Model
import torch
import torch.nn as nn
from transformers import BertModel

# The TextModel class definition has been moved to cell FADbM2wULsvC
# This cell is now empty or can be removed if not needed for other purposes.

In [57]:
# Vision Model
import torch
import torch.nn as nn
from torchvision import models

# The VisionModel class definition has been moved to cell dP-cJzC4NBN6
# This cell is now empty or can be removed if not needed for other purposes.

In [58]:
# Multimodal Fusion
import torch
import torch.nn as nn

class MultimodalFusion(nn.Module):
    def __init__(self):
        super(MultimodalFusion, self).__init__()

        self.classifier = nn.Sequential(
            nn.Linear(256 * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1)
        )

    def forward(self, audio_emb, text_emb, vision_emb):
        fused = torch.cat((audio_emb, text_emb, vision_emb), dim=1)
        return self.classifier(fused)


In [59]:
# Train Audio Model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Audio Model Definition (moved from cell CQO-NYthtQkp)
class AudioModel(nn.Module):
    def __init__(self, input_dim=40, hidden_dim=128, num_layers=2):
        super(AudioModel, self).__init__()

        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )

        self.fc = nn.Linear(hidden_dim * 2, 256)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h = torch.cat((h_n[-2], h_n[-1]), dim=1)
        return self.fc(h)

# ------------------ DEVICE ------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------ MODEL + CLASSIFIER ------------------
audio_model = AudioModel().to(device)
audio_classifier = nn.Sequential(
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 1)
).to(device)

# ------------------ OPTIMIZER & LOSS ------------------
optimizer = torch.optim.Adam(
    list(audio_model.parameters()) + list(audio_classifier.parameters()), lr=2e-4
)
criterion = nn.BCEWithLogitsLoss()

# ------------------ TRAIN LOOP ------------------
def train_one_epoch(dataloader):
    audio_model.train()
    audio_classifier.train()
    total_loss = 0
    for batch in dataloader:
        audio_x = batch["audio"].to(device)
        labels = batch["label"].float().to(device)

        audio_emb = audio_model(audio_x)
        outputs = audio_classifier(audio_emb).squeeze(1)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# ------------------ SAVE MODEL ------------------
def save_model():
    torch.save(audio_model.state_dict(), "audio_model.pt")
    torch.save(audio_classifier.state_dict(), "audio_classifier.pt")

# Call save_model to create the .pt files
save_model()

In [60]:
# Train Text Model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# TextModel Definition (moved from cell g2rBEVrbIItr)
from transformers import BertModel

class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()

        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 256)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        cls_embedding = outputs.last_hidden_state[:, 0, :]
        return self.fc(cls_embedding)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

text_model = TextModel().to(device)
text_classifier = nn.Sequential(
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 1)
).to(device)

optimizer = torch.optim.Adam(
    list(text_model.parameters()) + list(text_classifier.parameters()), lr=2e-4
)
criterion = nn.BCEWithLogitsLoss()

def train_one_epoch(dataloader):
    text_model.train()
    text_classifier.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].float().to(device)

        text_emb = text_model(input_ids, attention_mask)
        outputs = text_classifier(text_emb).squeeze(1)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

def save_model():
    torch.save(text_model.state_dict(), "text_model.pt")
    torch.save(text_classifier.state_dict(), "text_classifier.pt")

# Call save_model to create the .pt files
save_model()

In [61]:
#  Train Vision MOdel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
# from vision_model import VisionModel # This import is no longer needed

# VisionModel Definition (moved from cell npp2PRGVJQHx)
from torchvision import models

class VisionModel(nn.Module):
    def __init__(self):
        super(VisionModel, self).__init__()

        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()
        self.fc = nn.Linear(1280, 256)

    def forward(self, x):
        features = self.cnn(x)
        return self.fc(features)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vision_model = VisionModel().to(device)
vision_classifier = nn.Sequential(
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 1)
).to(device)

optimizer = torch.optim.Adam(
    list(vision_model.parameters()) + list(vision_classifier.parameters()), lr=2e-4
)
criterion = nn.BCEWithLogitsLoss()

def train_one_epoch(dataloader):
    vision_model.train()
    vision_classifier.train()
    total_loss = 0
    for batch in dataloader:
        images = batch["images"].to(device)
        labels = batch["label"].float().to(device)

        vision_emb = vision_model(images)
        outputs = vision_classifier(vision_emb).squeeze(1)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

def save_model():
    torch.save(vision_model.state_dict(), "vision_model.pt")
    torch.save(vision_classifier.state_dict(), "vision_classifier.pt")

# Call save_model to create the .pt files
save_model()

  warnings.warn(

  warnings.warn(msg)



In [62]:
# Train FUsion MOdel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Audio Model Definition (copied from W-lU2qGHJYXT)
class AudioModel(nn.Module):
    def __init__(self, input_dim=40, hidden_dim=128, num_layers=2):
        super(AudioModel, self).__init__()
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_dim * 2, 256)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h = torch.cat((h_n[-2], h_n[-1]), dim=1)
        return self.fc(h)

# TextModel Definition (copied from FADbM2wULsvC)
from transformers import BertModel

class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 256)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        return self.fc(cls_embedding)

# VisionModel Definition (copied from dP-cJzC4NBN6)
from torchvision import models

class VisionModel(nn.Module):
    def __init__(self):
        super(VisionModel, self).__init__()
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()
        self.fc = nn.Linear(1280, 256)

    def forward(self, x):
        features = self.cnn(x)
        return self.fc(features)

# MultimodalFusion Definition (copied from 1lUJQMhtNqWX)
class MultimodalFusion(nn.Module):
    def __init__(self):
        super(MultimodalFusion, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(256 * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1)
        )

    def forward(self, audio_emb, text_emb, vision_emb):
        fused = torch.cat((audio_emb, text_emb, vision_emb), dim=1)
        return self.classifier(fused)

# ------------------ DEVICE ------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------ LOAD PRETRAINED MODELS ------------------
audio_model = AudioModel().to(device)
text_model = TextModel().to(device)
vision_model = VisionModel().to(device)

audio_model.load_state_dict(torch.load("audio_model.pt"))
text_model.load_state_dict(torch.load("text_model.pt"))
vision_model.load_state_dict(torch.load("vision_model.pt"))

# Freeze pretrained models
for model in [audio_model, text_model, vision_model]:
    for param in model.parameters():
        param.requires_grad = False

# ------------------ FUSION NETWORK ------------------
fusion_model = MultimodalFusion().to(device)

# ------------------ OPTIMIZER & LOSS ------------------
optimizer = torch.optim.Adam(fusion_model.parameters(), lr=2e-4)
criterion = nn.BCEWithLogitsLoss()

# ------------------ TRAIN LOOP ------------------
def train_one_epoch(dataloader):
    fusion_model.train()
    total_loss = 0

    for batch in dataloader:
        # Load inputs
        audio_x = batch["audio"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        images = batch["images"].to(device)
        labels = batch["label"].float().to(device)

        # Get embeddings from frozen models
        audio_emb = audio_model(audio_x)
        text_emb = text_model(input_ids, attention_mask)
        vision_emb = vision_model(images)

        # Fusion network forward
        outputs = fusion_model(audio_emb, text_emb, vision_emb).squeeze(1)

        # Compute loss and update
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# ------------------ VALIDATION ------------------
def validate(dataloader):
    fusion_model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            audio_x = batch["audio"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["images"].to(device)
            labels = batch["label"].float().to(device)

            audio_emb = audio_model(audio_x)
            text_emb = text_model(input_ids, attention_mask)
            vision_emb = vision_model(images)

            outputs = fusion_model(audio_emb, text_emb, vision_emb).squeeze(1)
            loss = criterion(outputs, labels)

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels.bool()).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    accuracy = correct / total
    return total_loss / len(dataloader), accuracy

# ------------------ SAVE FUSION MODEL ------------------
def save_fusion():
    torch.save(fusion_model.state_dict(), "fusion_model.pt")

# Call save_fusion to create the .pt file
save_fusion()

In [63]:
# Install missing libraries
!pip install pydub
!pip install SpeechRecognition

import torch
import cv2
import os
import numpy as np
from PIL import Image
import librosa # For extract_audio_features
import speech_recognition as sr # For process_text
from pydub import AudioSegment # For process_audio
import dlib # For face detection in crop_align_face
from transformers import AutoTokenizer, AutoModel, BertModel # For TextModel and tokenize_text
from torchvision import models # For VisionModel
import shutil # For cleaning up temporary frames


# ------------------ CLASS DEFINITIONS (INLINED) ------------------
# Audio Model Definition (copied from W-lU2qGHJYXT)
class AudioModel(nn.Module):
    def __init__(self, input_dim=40, hidden_dim=128, num_layers=2):
        super(AudioModel, self).__init__()
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_dim * 2, 256)

    def forward(self, x):
        # x is expected to be (batch_size, seq_len, input_dim)
        _, (h_n, _) = self.lstm(x)
        h = torch.cat((h_n[-2], h_n[-1]), dim=1)
        return self.fc(h)

# TextModel Definition (copied from FADbM2wULsvC)
class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 256)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        return self.fc(cls_embedding)

# VisionModel Definition (copied from dP-cJzC4NBN6)
class VisionModel(nn.Module):
    def __init__(self):
        super(VisionModel, self).__init__()
        self.cnn = models.efficientnet_b0(pretrained=True)
        self.cnn.classifier = nn.Identity()
        self.fc = nn.Linear(1280, 256)

    def forward(self, x):
        features = self.cnn(x)
        return self.fc(features)

# MultimodalFusion Definition (copied from 1lUJQMhtNqWX)
class MultimodalFusion(nn.Module):
    def __init__(self):
        super(MultimodalFusion, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(256 * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1)
        )

    def forward(self, audio_emb, text_emb, vision_emb):
        fused = torch.cat((audio_emb, text_emb, vision_emb), dim=1)
        return self.classifier(fused)

# ------------------ HELPER FUNCTIONS (INLINED AND ADAPTED) ------------------

# Initialize dlib components for crop_align_face
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Function to crop and align face from an image (adapted to return PIL Image)
def crop_align_face(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not read image {image_path}. Skipping.")
        return []

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_detected = detector(gray)

    aligned_face_images = []
    for face in faces_detected:
        landmarks = predictor(gray, face)
        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)

        dy = right_eye[1] - left_eye[1]
        dx = right_eye[0] - left_eye[0]
        angle = np.degrees(np.arctan2(dy, dx))

        eyes_center = ((left_eye[0] + right_eye[0]) // 2,
                       (left_eye[1] + right_eye[1]) // 2)

        M = cv2.getRotationMatrix2D(eyes_center, angle, 1.0)
        aligned_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)

        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cropped_face = aligned_img[y:y+h, x:x+w]

        cropped_face_rgb = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)
        aligned_face_images.append(Image.fromarray(cropped_face_rgb))
    return aligned_face_images

# Extract audio features (adapted to return MFCC tensor)
def extract_audio_features(audio_path):
  y, sr = librosa.load(audio_path, sr = 16000)
  mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40) # n_mfcc matches AudioModel's input_dim
  # mfccs shape is (n_mfcc, time_frames). LSTM expects (seq_len, input_dim) after batch.
  # So transpose to (time_frames, n_mfcc)
  return torch.tensor(mfccs.T, dtype=torch.float32)

# Initialize tokenizer for tokenize_text
embed_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def tokenize_text(text):
    tokens = embed_tokenizer(text, return_tensors = "pt", truncation = True, padding = True)
    return tokens['input_ids'], tokens['attention_mask']


# ------------------ DEVICE ------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------ LOAD PRETRAINED MODELS ------------------
audio_model = AudioModel().to(device)
text_model = TextModel().to(device)
vision_model = VisionModel().to(device)
fusion_model = MultimodalFusion().to(device)

# Load state dicts
# Check if .pt files exist before loading, as they might not be generated if upstream cells failed
if os.path.exists("audio_model.pt"):
    audio_model.load_state_dict(torch.load("audio_model.pt"))
else:
    print("Warning: 'audio_model.pt' not found. Audio model might not be initialized with trained weights.")

if os.path.exists("text_model.pt"):
    text_model.load_state_dict(torch.load("text_model.pt"))
else:
    print("Warning: 'text_model.pt' not found. Text model might not be initialized with trained weights.")

if os.path.exists("vision_model.pt"):
    vision_model.load_state_dict(torch.load("vision_model.pt"))
else:
    print("Warning: 'vision_model.pt' not found. Vision model might not be initialized with trained weights.")

# fusion_model.pt is typically saved after training the fusion network
# If it's intended to be loaded here, it also needs to be saved after its training loop.
# Assuming it might not exist yet, we'll add a similar check.
if os.path.exists("fusion_model.pt"):
    fusion_model.load_state_dict(torch.load("fusion_model.pt"))
else:
    print("Warning: 'fusion_model.pt' not found. Fusion model might not be initialized with trained weights.")

for model in [audio_model, text_model, vision_model, fusion_model]:
    model.eval()
    for param in model.parameters():
        param.requires_grad = False

# ------------------ PREDICTION FUNCTION ------------------
def predict(audio_x=None, input_ids=None, attention_mask=None, images=None):
    with torch.no_grad():
        audio_emb = audio_model(audio_x.to(device).unsqueeze(0)) if audio_x is not None else None # Add batch dim for audio LSTM
        text_emb = text_model(input_ids.to(device), attention_mask.to(device)) if input_ids is not None else None
        vision_emb = vision_model(images.to(device)) if images is not None else None

        # Ensure all embeddings are available for multimodal fusion
        if audio_emb is None or text_emb is None or vision_emb is None:
            # This case should ideally not happen if all preprocessing steps succeed.
            # For a robust system, you might want to handle partial inputs.
            raise ValueError("All modalities (audio, text, vision) must be provided for prediction.")

        output = fusion_model(audio_emb, text_emb, vision_emb).squeeze(1)
        prob = torch.sigmoid(output).cpu().item()
    return prob

# ------------------ VIDEO PROCESSING ------------------
def process_video(video_path, tmp_folder="tmp_frames"):
    os.makedirs(tmp_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    frames_paths = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(tmp_folder, f"frame_{frame_count:04d}.jpg") # Added padding to frame count
        cv2.imwrite(frame_path, frame)
        frames_paths.append(frame_path)
        frame_count += 1

    cap.release()
    return frames_paths

# ------------------ AUDIO PROCESSING ------------------
def process_audio(video_path, audio_output_path="temp_audio.wav"):
    # Use moviepy.editor to extract audio
    clip = mp.VideoFileClip(video_path)
    clip.audio.write_audio_file(audio_output_path, fps=16000)
    clip.close()

    features = extract_audio_features(audio_output_path)
    return features

# ------------------ TEXT PROCESSING ------------------
def process_text(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = r.record(source)
        text = r.recognize_google(audio_data)
    input_ids, attention_mask = tokenize_text(text)
    return input_ids, attention_mask

# ------------------ VISUAL PROCESSING ------------------
def process_frames(frame_paths):
    aligned_faces = []
    for frame_path in frame_paths:
        faces = crop_align_face(frame_path)  # Now returns list of PIL images
        aligned_faces.extend(faces)

    if not aligned_faces:
        print("No faces detected in frames.")
        return torch.tensor([])

    # Resize and convert to tensor (assuming all images are 224x224 for EfficientNet_b0)
    # EfficientNet_b0 expects (batch, channels, height, width)
    transformed_images = []
    for face_img in aligned_faces:
        # Resize to 224x224 as expected by EfficientNet_b0
        face_img = face_img.resize((224, 224))
        img_array = np.array(face_img).transpose(2,0,1) / 255.0 # (H,W,C) to (C,H,W), normalize
        transformed_images.append(img_array)

    images_tensor = torch.tensor(transformed_images, dtype=torch.float32)
    return images_tensor

# ------------------ MAIN PREDICTION FUNCTION ------------------
def main(video_file):
    tmp_folder = "tmp_prediction_assets"
    os.makedirs(tmp_folder, exist_ok=True)
    audio_output_path = os.path.join(tmp_folder, "temp_audio.wav")
    frames_output_dir = os.path.join(tmp_folder, "frames")

    try:
        # Process video frames
        print("Processing video frames...")
        frame_paths = process_video(video_file, tmp_folder=frames_output_dir)
        images_tensor = process_frames(frame_paths)
        # Average vision embeddings or take first if multiple frames/faces or handle as sequence
        if images_tensor.numel() == 0: # Check if tensor is empty
             vision_input = None
             print("Warning: No valid vision input generated.")
        else:
            # For simplicity, let's take the first image if multiple frames/faces are processed.
            # A more robust approach might involve averaging embeddings or using a sequence model.
            vision_input = images_tensor[0:1] # Take first image, keep batch dimension

        # Process audio
        print("Extracting audio...")
        audio_input = process_audio(video_file, audio_output_path)
        # AudioModel expects (batch_size, seq_len, input_dim). audio_input is (seq_len, input_dim)
        # It will be unsqueezed in predict()

        # Process text
        print("Transcribing audio to text and extracting text features...")
        input_ids, attention_mask = process_text(audio_output_path)
        # input_ids and attention_mask are already (batch_size, seq_len)

        # Predict
        print("Making prediction...")
        prob = predict(audio_x=audio_input, input_ids=input_ids, attention_mask=attention_mask, images=vision_input)
        print(f"Deepfake Probability: {prob:.4f}")

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
    finally:
        # Clean up temporary files
        if os.path.exists(tmp_folder):
            shutil.rmtree(tmp_folder)
            print(f"Cleaned up temporary folder: {tmp_folder}")

# ------------------ RUN ------------------
if __name__ == "__main__":
    # IMPORTANT: Ensure a valid video file is present in 'Data/videos' for this to work.
    # Replace "dummy_video.mp4" with the actual path to your video file.
    video_file = "Data/videos/your_video.mp4"  # <--- REPLACE 'your_video.mp4' with your actual video filename (e.g., 'my_real_video.mp4')

    print("Please upload a real video file to 'Data/videos' and update 'video_file' variable for actual prediction.")
    print(f"Currently trying to process: {video_file}")
    main(video_file)

Please upload a real video file to 'Data/videos' and update 'video_file' variable for actual prediction.
Currently trying to process: Data/videos/your_video.mp4
Processing video frames...
No faces detected in frames.
Extracting audio...
An error occurred during prediction: MoviePy error: the file Data/videos/your_video.mp4 could not be found!
Please check that you entered the correct path.
Cleaned up temporary folder: tmp_prediction_assets
